In [2]:
import os
import sys
import json
from sklearn.model_selection import train_test_split
from mmengine.config import Config
from mmengine.runner import Runner
from mmengine.hooks import EarlyStoppingHook
from mmengine.visualization import Visualizer, WandbVisBackend
from mmdet.utils import setup_cache_size_limit_of_dynamo
import datetime
import wandb

# mmdetection을 시스템 경로에 추가
sys.path.insert(0, "../mmdetection")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 사용자 맞춤형 학습 설정
config_path = './configs/dino/dino-5scale_swin-l_8xb2-12e_coco.py'  # DINO 설정 파일 경로
work_dir = './work_dirs/dino_custom'  # 로그와 모델을 저장할 디렉토리 경로
train_data_root = '/data/ephemeral/home/dataset/'  # 학습 데이터 경로
original_ann_file = '/data/ephemeral/home/dataset/train.json'  # 전체 데이터 어노테이션 파일 경로

In [4]:
# 학습 및 검증 데이터셋 나누기
with open(original_ann_file, 'r') as f:
    annotations = json.load(f)

# 이미지 ID 리스트 가져오기
image_ids = [image['id'] for image in annotations['images']]

# 학습 및 검증 셋으로 분할 (80% 학습, 20% 검증)
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

# 학습 및 검증 데이터 어노테이션 생성
train_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in train_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in train_ids],
    'categories': annotations['categories']
}
val_annotations = {
    'images': [img for img in annotations['images'] if img['id'] in val_ids],
    'annotations': [ann for ann in annotations['annotations'] if ann['image_id'] in val_ids],
    'categories': annotations['categories']
}

# 분할된 어노테이션을 파일로 저장
train_ann_file = '/data/ephemeral/home/dataset/train_split.json'
val_ann_file = '/data/ephemeral/home/dataset/val_split.json'

with open(train_ann_file, 'w') as f:
    json.dump(train_annotations, f)
with open(val_ann_file, 'w') as f:
    json.dump(val_annotations, f)

In [5]:
# 어노테이션 파일 수정: category_id를 1부터 시작하도록 조정
def increment_category_id(annotation_file):
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # 'categories' 섹션의 'id'를 1부터 시작하도록 수정
    id_mapping = {}
    for category in data['categories']:
        old_id = category['id']
        new_id = old_id + 1
        id_mapping[old_id] = new_id
        category['id'] = new_id

    # 'annotations' 섹션의 'category_id'를 매핑에 따라 수정
    for ann in data['annotations']:
        old_cat_id = ann['category_id']
        if old_cat_id in id_mapping:
            ann['category_id'] = id_mapping[old_cat_id]
        else:
            print(f"Warning: annotation id {ann['id']} has invalid category_id {old_cat_id}")

    # 수정된 데이터를 원래 파일에 저장
    with open(annotation_file, 'w') as f:
        json.dump(data, f, indent=4)

# 어노테이션 파일에 category_id를 1부터 시작하도록 수정
increment_category_id(train_ann_file)
increment_category_id(val_ann_file)


In [6]:
# 클래스 설정
classes = (
    "General trash", "Paper", "Paper pack", "Metal", "Glass",
    "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"
)
num_classes = len(classes)  # 10

In [7]:
# 설정 파일 로드
cfg = Config.fromfile(config_path)

# 작업 디렉토리 수정
cfg.work_dir = work_dir  # 로그와 모델 저장을 위한 작업 디렉토리 수정

# EarlyStoppingHook 추가
early_stopping_hook = dict(
    type='EarlyStoppingHook',
    monitor='bbox_mAP',
    rule='greater',
    min_delta=0.001,
    patience=5,
    check_finite=True,
    stopping_threshold=None
)

# cfg.default_hooks에 EarlyStoppingHook 추가
cfg.default_hooks.update(
    early_stopping=early_stopping_hook
)

# WandB 초기화 (Runner 전에)
run_name = f'dino-5scale_last_layer_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'
wandb.init(
    project='Object_detection',
    name=run_name,
    config=cfg.to_dict(),
    allow_val_change=True,
    reinit=True
)
print(f"WandB Run Initialized: {run_name}")

# WandbVisBackend 설정
wandb_vis_backend = dict(
    type='WandbVisBackend',
    save_dir=cfg.work_dir,  # 저장할 디렉토리
    init_kwargs=dict(
        project='Object_detection',  # WandB 프로젝트 이름
        name=run_name,                # 고유한 실행 이름
        allow_val_change=True         # 설정 값 변경 허용
    ),
    define_metric_cfg=None,
    commit=True,
    log_code_name=None,
    watch_kwargs=None
)

# Visualizer 설정을 Runner의 visualizer 필드로 추가
cfg.visualizer = dict(
    type='Visualizer',
    vis_backends=[
        wandb_vis_backend  # WandbVisBackend 추가
    ],
    name='visualizer'  # Visualizer 이름 (옵션)
)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: youngtae0818 (youngtae0818-naver). Use `wandb login --relogin` to force relogin


WandB Run Initialized: dino-5scale_last_layer_20241006_234552


In [8]:
# 데이터셋 루트 경로 수정
cfg.train_dataloader.dataset.ann_file = train_ann_file  # 학습 데이터 어노테이션 파일 경로 수정
cfg.train_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 학습 데이터 이미지 경로 수정

cfg.val_dataloader.dataset.ann_file = val_ann_file  # 검증 데이터 어노테이션 파일 경로 수정
cfg.val_dataloader.dataset.data_prefix = dict(img=train_data_root)  # 검증 데이터 이미지 경로 수정

# 클래스 설정을 데이터셋의 metainfo에 추가
cfg.train_dataloader.dataset.metainfo = dict(classes=classes)  # 학습 데이터셋 클래스 설정
cfg.val_dataloader.dataset.metainfo = dict(classes=classes)    # 검증 데이터셋 클래스 설정

# 클래스 수 수정
cfg.model.bbox_head.num_classes = num_classes  # 모델의 클래스 수 수정

In [9]:
# # **모델의 Backbone 동결**
# cfg.model.backbone.frozen_stages = 4  # Backbone의 모든 스테이지를 동결

# **옵티마이저 설정 조정 (옵션)**
# Backbone의 학습률을 설정할 필요가 없으므로 paramwise_cfg를 제거하거나 조정
cfg.optim_wrapper.paramwise_cfg = None  # Backbone의 lr_mult 제거

In [10]:
# 자동 혼합 정밀도 학습 사용 설정
use_amp = True  # 자동 혼합 정밀도(AMP) 학습 사용 여부
if use_amp:
    cfg.optim_wrapper.type = 'AmpOptimWrapper'
    cfg.optim_wrapper.loss_scale = 'dynamic'

# 학습률 자동 스케일링 설정
auto_scale_lr = False  # 학습률 자동 스케일링 사용 여부
if auto_scale_lr:
    if 'auto_scale_lr' in cfg and 'enable' in cfg.auto_scale_lr and 'base_batch_size' in cfg.auto_scale_lr:
        cfg.auto_scale_lr.enable = True
    else:
        raise RuntimeError('설정 파일에 "auto_scale_lr" 또는 필요한 키가 없습니다.')


In [11]:
# 데이터 로더 설정 최적화
cfg.train_dataloader.batch_size = 1  # 배치 사이즈를 1로 줄임
cfg.val_dataloader.batch_size = 1

cfg.train_dataloader.num_workers = 2  # 워커 수 줄이기
cfg.val_dataloader.num_workers = 2

# Prefetch factor와 persistent_workers 설정
cfg.train_dataloader.prefetch_factor = 2
cfg.train_dataloader.persistent_workers = False
cfg.val_dataloader.prefetch_factor = 2
cfg.val_dataloader.persistent_workers = False

In [12]:
# 학습 재개 설정
resume_training = False  # 학습 재개 여부 설정 (True로 설정 시 학습 재개)
resume_checkpoint_path = None  # 학습 재개 시 체크포인트 경로 지정
if resume_training:
    if resume_checkpoint_path is None:
        cfg.resume = True
        cfg.load_from = None
    else:
        cfg.resume = True
        cfg.load_from = resume_checkpoint_path

In [13]:
# 반복적인 컴파일 횟수를 줄여 학습 속도 향상
setup_cache_size_limit_of_dynamo()

In [14]:
# 평가 설정 추가 및 수정
cfg.evaluation = dict(interval=1, metric='bbox', save_best='auto')

# 평가자(evaluator) 설정 수정
if hasattr(cfg, 'val_evaluator'):
    # 평가자가 딕셔너리 또는 리스트로 정의되어 있을 경우
    if isinstance(cfg.val_evaluator, dict):
        cfg.val_evaluator.ann_file = val_ann_file
    elif isinstance(cfg.val_evaluator, list):
        for evaluator in cfg.val_evaluator:
            evaluator['ann_file'] = val_ann_file
elif 'evaluation' in cfg:
    # 평가 섹션 내에 ann_file을 추가
    cfg.evaluation['ann_file'] = val_ann_file


In [15]:
# # 모든 레이어를 동결
# for name, param in runner.model.named_parameters():
#     param.requires_grad = False

# # bbox_head만 unfreeze
# for param in runner.model.bbox_head.parameters():
#     param.requires_grad = True

# # 백본의 마지막 두 레이어 unfreeze
# for name, param in runner.model.backbone.named_parameters():
#     if 'layers.2' in name or 'layers.3' in name:
#         param.requires_grad = True


In [16]:
# 설정 파일을 통해 러너 생성
runner = Runner.from_cfg(cfg)

# 학습 시작 전에 검증 어노테이션 파일 존재 여부 확인
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f'Validation annotation file not found: {val_ann_file}')

# 학습 시작
runner.train()

10/06 23:45:55 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.13 (main, Sep 11 2023, 13:44:35) [GCC 11.2.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 112301514
    GPU 0: Tesla V100-SXM2-32GB
    CUDA_HOME: None
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 1.12.1+cu116
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.6
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-ge

10/06 23:46:07 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/06 23:46:07 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
(LOWEST      ) EarlyStoppingHook                  
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_i

/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/positional_encoding.py:103: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature**(2 * (dim_t // 2) / self.num_feats)
/opt/conda/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/data/ephemeral/home/baseline/mmdetection/mmdet/models/layers/transformer/utils.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change 

KeyboardInterrupt: 

: 